In [1]:
from    ctypes import *
import ctypes
cdll.LoadLibrary("libc.so.6") # load the C dynamiclinked library shared object 6 as advised by ctypes docs

from    scipy import sparse
import  numpy as np
import  random



'''# speed comparison of four approaches    '''
'''    # native python libraries                        '''
'''    # specialist libraries                                   '''
'''    # ctype method calls into C                                      '''
'''    # wholesale modular call into C from compiled library                    '''
'''                                                                                 '''
'''# further optimisations maybe possible with the four approaches                      '''
'''# note that mutate request internally casts the numpy population into a list of lists    '''

class MutateRequest:
    def __init__(self, population):
        
        # I thought it was fairer to start with the native list object and require modules to promote to array
        # than have "naive" modules demoting nd arrays back down to lists
        number_of_mutations = get_number_of_mutations()
        self.population     = population
        self.population     = [list(algo) for algo in population] # see explanation of choice of list not ndarray
        self.rows_algos     = len(population)
        self.columns_genes  = len(population[0])
        self.number_of_mutations = number_of_mutations

class TellObject:
    def __init__(self, population, base_mutants, numpy_mutants, ctype_mutants, pygad_mutants):
        self.population     = population
        self.base_mutants   = base_mutants
        self.numpy_mutants  = numpy_mutants
        self.ctype_mutants  = ctype_mutants
        self.pygad_mutants  = pygad_mutants

def get_random_mutation_min_val():
    return -2

def get_random_mutation_max_val():
    return 2
        
def tell(tell_object):
    print('\noriginal population\n', np.around(np.asarray(tell_object.population), decimals=4))
    print('\nmutation by base libraries')
    for i in range(len(tell_object.base_mutants)):
          print(np.around(np.asarray(tell_object.base_mutants[i]),decimals=4))
    print('\nmutation by numpy/scipy libraries\n', np.around(np.asarray(tell_object.numpy_mutants), decimals=4))
    print('\nmutation by pygad function\n', np.around(np.asarray(tell_object.pygad_mutants), decimals=4))
    print('\nmutation by ctype library\n', np.around(np.asarray(tell_object.ctype_mutants), decimals=4))

def get_number_of_mutations():
    number_of_mutations = 1
    return number_of_mutations

def build_population():
    low         = -1
    high        =  1
    cohort_size = (4, 8) #  rows_algos, columns_genes
    population  = np.random.uniform(low, high, size=cohort_size)
    return population

def base_mutate(mutate_request):
    ''' does not use numpy; only uses naive base library - math, itertools, random etc are allowed'''
    population          = mutate_request.population
    number_of_mutations = mutate_request.number_of_mutations
    rows_algos          = mutate_request.rows_algos
    columns_genes       = mutate_request.columns_genes
    mutants             = population
    
    # arbitrary loop nesting order here - this is best for 
    for mutation in range(number_of_mutations):
        for algo in mutants:
            random_place    = random.randrange(columns_genes)
            starting_gene   = algo[random_place]
            a_float         = random.random()
            algo.pop(random_place)
            mutant_gene     = starting_gene + a_float
            algo.insert(random_place, mutant_gene)
    return mutants

def numpy_mutate(mutate_request):
    population          = mutate_request.population
    population          = np.asarray(population)
    
    number_of_mutations = mutate_request.number_of_mutations
    all_loci            = mutate_request.rows_algos * mutate_request.columns_genes
    
    mutation_rate       = number_of_mutations/mutate_request.columns_genes
    
    mutation_matrix = sparse.random(population.shape[0], population.shape[1], density=mutation_rate)
    pre_mutants     = np.copy(population)
    mutants         = np.add(pre_mutants, mutation_matrix.todense())
    return mutants


In [2]:
x = (ctypes.c_ulong*5)(1,2,3,4,5)

In [3]:
vars(x)

{}

In [4]:
print(ctypes.cast(x, ctypes.POINTER(ctypes.c_ulong)))

In [11]:
def ctype_mutate(mutate_request):
    
    random_mutation_min_val = get_random_mutation_min_val()
    random_mutation_max_val = get_random_mutation_max_val()
    gap                     = random_mutation_max_val - random_mutation_min_val
    
    libc        = CDLL("libc.so.6")
    c_RAND_MAX  = 2147483647 # as if libc.RAND_MAX returns 2 ^ 32-1 but instead it throws undefined symobl
    c_rand      = libc.rand
    
    population          = mutate_request.population
    flat_population     = list(np.ravel(np.asarray(population)))
    population_length   = len(flat_population)
    mutations_needed    = mutate_request.rows_algos * mutate_request.number_of_mutations

    CFlatPopulation     = c_longdouble * population_length
    c_flat_population   = CFlatPopulation(*flat_population) # * means unpack
    
    for i in range(population_length):
        this_locus_pointer  = pointer(c_longdouble(c_flat_population[i])) # pointer to ith element
        #this_locus_pointer  = POINTER(c_longdouble(c_flat_population[i])) # pointer to ith element
        print('\nthis_locus_pointer.contents:\t', this_locus_pointer.contents)

    for needed_mutation in range(mutations_needed):
        index       = c_int(c_int(population_length) * c_int(c_rand())/c_int(c_RAND_MAX))
        #mutation    = c_longdouble(random_mutation_min_val + gap*(c_rand()/c_RAND_MAX))
        #this_locus  = pointer(c_longdouble(c_flat_population[index]))
        #this_gene   = pointer(c_longdouble(c_flat_population[index])).contents
        #print('\nindex\t', index, 'mutation\t', mutation, 'this_locus\t', this_locus, 'old_gene\t', this_gene)
        

        
        
    #print('\npy_index_stack\n', py_index_stack)
    #print('\npy_mutations_stack\n', py_mutations_stack)
    
    #CIndexStack         = c_int        * mutations_needed
    #CMutationsStack     = c_longdouble * mutations_needed
    
    #for i in range(mutations_needed):
        
        
    
    
    
    
    
    
    
    #py_index_stack_pointer      = pointer(py_index_stack)
    #py_mutations_stack_pointer  = pointer(py_mutations_stack)
    
    #print('\n py_index_stack_pointer in ctype mutate scope\n', py_index_stack_pointer)
    #print('\n py_mutations_stack_pointer in ctype mutate scope\n', py_mutations_stack_pointer)
    

    
    #     use pointer + random int of index stack[i] to retrieve, increment and replace double at [i]
    
    # whap the quickly incremented C array back into pytohn land
    # reshape that into population ndarray shape
    # bosh
    mutants = population
    return mutants

def test_ctype_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = ctype_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      list),  'Error: should be an np array'
    assert isinstance(mutants[0],   list),  'Error: should be an np array'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    
test_ctype_mutate()


this_locus_pointer.contents:	 c_longdouble(0.9549690640089585)

this_locus_pointer.contents:	 c_longdouble(0.444723882195583)

this_locus_pointer.contents:	 c_longdouble(-0.8443044344004766)

this_locus_pointer.contents:	 c_longdouble(-0.13918018960654832)

this_locus_pointer.contents:	 c_longdouble(0.8042681282695889)

this_locus_pointer.contents:	 c_longdouble(0.22201060365870173)

this_locus_pointer.contents:	 c_longdouble(-0.17104413980832112)

this_locus_pointer.contents:	 c_longdouble(-0.291142357486714)

this_locus_pointer.contents:	 c_longdouble(-0.24166879948512077)

this_locus_pointer.contents:	 c_longdouble(-0.5471210077854611)

this_locus_pointer.contents:	 c_longdouble(0.43288166360882085)

this_locus_pointer.contents:	 c_longdouble(0.6228562279576517)

this_locus_pointer.contents:	 c_longdouble(0.8052018856393457)

this_locus_pointer.contents:	 c_longdouble(-0.473808695663539)

this_locus_pointer.contents:	 c_longdouble(-0.13882232888070112)

this_locus_pointer.contents:

TypeError: unsupported operand type(s) for *: 'c_int' and 'c_int'

In [8]:
def pygad_mutate(mutate_request):
    # 1. removed dependency on gene_type
    # 2. all modules for these time trials start with a list because they refer to the MutateRequest
    #    and the mutate reqeust offers a list as the most basic thing
    #    so if this module seems handicapped by e.g. the need to retype offspring as an array then
    #    people are welcome to change the interface, but it just passes that handicap elsewhere

    offspring                = np.asarray(mutate_request.population)
    num_genes                = mutate_request.columns_genes
    mutation_num_genes       = mutate_request.number_of_mutations
    random_mutation_min_val  = get_random_mutation_min_val()
    random_mutation_max_val  = get_random_mutation_max_val()
    
    for offspring_idx in range(offspring.shape[0]):
        mutation_indices = np.array(random.sample(range(0, num_genes), mutation_num_genes))
        for gene_idx in mutation_indices:
            random_value = np.random.uniform(low=random_mutation_min_val, 
                                            high=random_mutation_max_val, 
                                            size=1)
            offspring[offspring_idx, gene_idx] = offspring[offspring_idx, gene_idx] + random_value
    return offspring

In [9]:
if __name__ == '__main__':
    
    population          = build_population()
    mutate_request      = MutateRequest(population)
    
    base_mutants        =  base_mutate(mutate_request)
    numpy_mutants       = numpy_mutate(mutate_request)
    ctype_mutants       = ctype_mutate(mutate_request)
    pygad_mutants       = pygad_mutate(mutate_request)
    
    that_which_to_tell = TellObject(population, base_mutants, numpy_mutants, ctype_mutants, pygad_mutants)
    tell(that_which_to_tell)
    


py_index_stack
 [7497, 2941, 5844, 1523, 1254, 1641, 745, 525, 1762, 7977, 6565, 6394, 934, 5202, 699, 4614, 5733, 519, 9999, 8899, 9977, 8705, 41, 5938, 1631, 9130, 3590, 5794, 6873, 5308, 3042, 5769, 7478, 354, 8332, 8732, 9794, 9033, 6668, 1639, 8889, 6497, 6294, 7006, 3287, 741, 2236, 5106, 2800, 7192, 4714, 9443, 3363, 4345, 3449, 8332, 6754, 4819, 7120, 6218]

py_mutations_stack
 [-0.5253458332854071, -1.0709538455451624, -1.0223490572638574, 0.9285940634685539, 1.1738815527287692, 0.9802855565120865, 1.8004161267543286, 0.08625351921015101, -1.039750510379556, 0.9306176467475562, 1.869620554088438, 1.0389393675322363, -1.4603903533240734, -1.6870714331451204, -1.1813796549948767, 1.2787091207125734, 1.0223233415849151, -1.3687714856903868, -1.1826855573722561, -1.4981260967897838, -1.7837696893996418, -1.7106848022484615, 1.6922765093353935, -1.278510937131248, -0.43323907741961953, 1.2787806108960793, 0.20994008994192814, -0.18969661658149994, -1.601439745911136, 1.02917532950

In [10]:
'''
# build popluation
'''

def test_build_population():
    population = build_population()
    assert isinstance(population,       np.ndarray),    'Error: should be array of arrays'
    assert isinstance(population[0],    np.ndarray),    'Error: should be an array'
    assert isinstance(population[0][0], np.float64),    'Error: should be a float'
    print('pass')

'''
# base mutate
'''    

def test_base_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = base_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      list),  'Error: should be a list'
    assert isinstance(mutants[0],   list),  'Error: should be a list'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    
    
'''
# ctype mutate
'''

# goes here


'''
# numpy mutate
'''

def test_numpy_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = numpy_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0],   np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0][0],np.matrix), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    
'''
# pygad mutate
'''
    
def test_pygad_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = pygad_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0],   np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    
'''
# tests per se
'''

def tests():
    
    test_base_mutate()
    test_build_population()
    #test_ctype_mutate()
    test_numpy_mutate()
    test_pygad_mutate() 
    print('\nwell done passed all active tests - stick to TDD even where you are excited.\n')
    
tests()

pass
pass
pass
pass

well done passed all active tests - stick to TDD even where you are excited.



In [11]:
'''# Note

 class ctypes.Array(*args)

    Abstract base class for arrays.

    The recommended way to create concrete array types is by multiplying any ctypes data type
    with a positive integer. Alternatively, you can subclass this type and define _length_ 
    and _type_ class variables. Array elements can be read and written using standard subscript 
    and slice accesses; for slice reads, the resulting object is not itself an Array.

'''

'# Note\n\n class ctypes.Array(*args)\n\n    Abstract base class for arrays.\n\n    The recommended way to create concrete array types is by multiplying any ctypes data type\n    with a positive integer. Alternatively, you can subclass this type and define _length_ \n    and _type_ class variables. Array elements can be read and written using standard subscript \n    and slice accesses; for slice reads, the resulting object is not itself an Array.\n\n'

In [12]:
cdll.LoadLibrary("libc.so.6")

<CDLL 'libc.so.6', handle 7ff69d947500 at 0x7ff6982c8b90>

In [13]:
libc = CDLL("libc.so.6")

In [14]:
c_wchar_p("Hello, World")

c_wchar_p(140696823468848)

In [15]:
printf = libc.printf

In [16]:
printf()

6

In [17]:
printf("Hello World")

1

In [18]:
s = "Hello, World"

In [19]:
c_s = c_wchar_p(s)

In [20]:
printf(b"Hello, %s\n", b"World!")

14

In [21]:
print(c_s.value)

Hello, World


In [22]:
print(c_s)

c_wchar_p(140696821942704)


In [23]:
print(s)                # first object is unchanged

Hello, World


In [24]:
printf(b"An int %d, a double %f\n", 1234, c_double(3.14))

31

In [25]:
strchr = libc.strchr

In [26]:
strchr(b"abcdef", ord("d"))  

-1741493501

In [27]:
i = c_int()

In [28]:
f = c_float()

In [29]:
s = create_string_buffer(b'\000' * 32)

In [30]:
print(i.value, f.value, repr(s.value))

0 0.0 b''


In [31]:
libc.sscanf(b"1 3.14 Hello", b"%d %f %s", byref(i), byref(f), s)

3

In [32]:
print(i.value, f.value, repr(s.value))

1 3.140000104904175 b'Hello'


In [33]:
class POINT(Structure):
    _fields_ = [("x", c_int),
                ("y", c_int)]

In [34]:
point = POINT(10, 20)

In [35]:
print(point.x, point.y)

10 20


In [31]:
point = POINT(y=5)

In [32]:
print(point.x, point.y)

0 5


In [33]:
POINT(1, 2)

In [34]:
POINT(-1, 2)

In [35]:
#POINT(1, 2.0001)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-120-a3808173cd67> in <module>
----> 1 POINT(1, 2.0001)

TypeError: int expected instead of float


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-120-a3808173cd67> in <module>\n----> 1 POINT(1, 2.0001)\n\nTypeError: int expected instead of float\n\n\n'

In [36]:
# POINT(1, -2.0001)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-121-bfc065603ff4> in <module>
----> 1 POINT(1, -2.0001)

TypeError: int expected instead of float


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-121-bfc065603ff4> in <module>\n----> 1 POINT(1, -2.0001)\n\nTypeError: int expected instead of float\n\n\n'

In [37]:
POINT(1, 2)

In [38]:
class RECT(Structure):
    _fields_ = [("upperleft", POINT),
                ("lowerright", POINT)]

In [39]:
rc = RECT(point)

In [40]:
print(rc.upperleft.x, rc.upperleft.y)

10 20


In [41]:
print(rc.lowerright.x, rc.lowerright.y)

0 0


In [42]:
TenPointsArrayType = POINT * 10

In [43]:
r = RECT(POINT(1, 2), POINT(3, 4))

In [44]:
r = RECT((1, 2), (3, 4))

In [45]:
print(POINT.x)

<Field type=c_int, ofs=0, size=4>


In [46]:
print(POINT.y)

<Field type=c_int, ofs=4, size=4>


In [47]:
class Int(Structure):
    _fields_ = [("first_16", c_int, 16),
                ("second_16", c_int, 16)]

In [48]:
print(Int.first_16)

<Field type=c_int, ofs=0:0, bits=16>


In [49]:
print(Int.second_16)

<Field type=c_int, ofs=0:16, bits=16>


In [50]:
class POINT(Structure):
    _fields_ = ("x", c_int), ("y", c_int)

In [51]:
class MyStruct(Structure):
    _fields_ = [("a", c_int),
                ("b", c_float),
                ("point_array", POINT * 4)]

In [52]:
print(len(MyStruct().point_array))

4


In [53]:
arr = TenPointsArrayType()

In [54]:
for pt in arr:
    print(pt.x, pt.y)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


In [55]:
TenIntegers = c_int * 10

In [56]:
ii = TenIntegers(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [57]:
print(ii)

In [58]:
for i in ii: print(i, end=" ")

1 2 3 4 5 6 7 8 9 10 

In [61]:
i = c_int(42)

In [62]:
pi = pointer(i)

In [63]:
pi.contents

c_int(42)

In [62]:
pi.contents is i

False

In [63]:
pi.contents is pi.contents

False

In [64]:
i = c_int(99)

In [65]:
pi.contents = i

In [66]:
pi.contents

c_int(99)

In [67]:
pi[0]

99

In [68]:
print(i)

c_int(99)


In [69]:
pi[0] = 22

In [70]:
print(i)

c_int(22)


In [71]:
PI = POINTER(c_int)

In [72]:
PI

importlib._bootstrap.LP_c_int

In [73]:
# PI(42)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-52-622c40ff30d9> in <module>
----> 1 PI(42)

TypeError: expected c_int instead of int


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-52-622c40ff30d9> in <module>\n----> 1 PI(42)\n\nTypeError: expected c_int instead of int\n\n\n'

In [74]:
PI(c_int(42))

In [75]:
null_ptr = POINTER(c_int)()

In [76]:
print(bool(null_ptr))

False


In [77]:
class Bar(Structure):
    _fields_ = [("count", c_int), ("values", POINTER(c_int))]

In [78]:
bar = Bar()

In [79]:
bar.values = (c_int * 3)(1, 2, 3)

In [80]:
bar.count = 3

In [81]:
for i in range(bar.count):
    print(bar.values[i])

1
2
3


In [82]:
IntArray5 = c_int * 5

In [83]:
ia = IntArray5(5, 1, 7, 33, 99)

In [84]:
qsort = libc.qsort

In [85]:
qsort.restype = None

In [86]:
CMPFUNC = CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))



In [87]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return 0

In [88]:
cmp_func = CMPFUNC(py_cmp_func)

In [89]:
qsort(ia, len(ia), sizeof(c_int), cmp_func) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 5 7
py_cmp_func 1 7


In [90]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]

In [91]:
qsort(ia, len(ia), sizeof(c_int), CMPFUNC(py_cmp_func)) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 1 7
py_cmp_func 5 7


In [92]:
for i in ia: print(i, end=" ")




1 5 7 33 99 

In [93]:
@CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]

In [94]:
qsort(ia, len(ia), sizeof(c_int), py_cmp_func)

py_cmp_func 1 5
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 1 7
py_cmp_func 5 7


In [95]:
short_array = (c_short * 4)()

In [96]:
print(sizeof(short_array))

8


In [97]:
#resize(short_array, 4)

'''
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-207-5d3303a611a5> in <module>
----> 1 resize(short_array, 4)
      2 
      3 

ValueError: minimum size is 8

'''

'\n---------------------------------------------------------------------------\nValueError                                Traceback (most recent call last)\n<ipython-input-207-5d3303a611a5> in <module>\n----> 1 resize(short_array, 4)\n      2 \n      3 \n\nValueError: minimum size is 8\n\n'

In [98]:
resize(short_array, 32)

In [99]:
sizeof(short_array)

32

In [100]:
sizeof(type(short_array))

8